# 평균 제곱 오차 ( MSE = Mean Squared Error)
$$
E = \frac{1}{n}\sum_{k}(y_k-t_k)^2
$$

* $y_k$는 예측값을 의미한다. ($\hat y$)
* $t_k$는 정답(타깃값)을 의미한다.

In [4]:
import numpy as np

y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

위의 'y'는 softmax 의 결과물 로써, 정답이 2일 확률을 60%로 추정하고 있다.
't'는 원핫인코딩된 정답 레이블 이다.

In [5]:
def mse(y,t):
    return np.sum((y-t)**2)/2

In [7]:
print('정답을 2로 추정했을 때의 mse 값 : {:.3f}'.format(mse(np.array(y),np.array(t))))

정답을 2로 추정했을 때의 mse 값 : 0.098


In [8]:
# 일부로 오답 예측을 만들어서 MSE 값을 측정하기. 예측값을 7이라고 가정한 경우로 테스트

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print('정답을 7로 추정했을 때의 mse 값 : {:.3f}'.format(mse(np.array(y),np.array(t))))

정답을 7로 추정했을 때의 mse 값 : 0.598


# 교차 엔트로피 ( CEE - Cross Entropy Error)

$$
E = -\sum_{k}t_klog\;y_k
$$

- 원핫인코딩 된 t 가 곱해지기 떄문에 , 정답이 아닌 타깃은 신경을 전혀 쓰지 않는다.

In [14]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum( t * np.log( y + delta))

# 아주 작은 값인 delta를 더하는 이유는 np.log 함수에 0이 들어가면 
# 마이너스 무한대를 나타나게 되기 때문에 y에 아주 작은 값인 delta를
# 더해서 np.log 함수의 결과물이 마이너스 무한대가 되는 것을 방지한다.

In [15]:
# 정답은 2
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# 정답이 2일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print("정답을 2로 추정했을 때의 CEE값 : {:.3f}".format(cross_entropy_error(np.array(y), np.array(t))))
# 정답이 7일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print("정답을 7로 추정했을 때의 CEE값 : {:.3f}".format(cross_entropy_error(np.array(y), np.array(t))))

정답을 2로 추정했을 때의 CEE값 : 0.511
정답을 7로 추정했을 때의 CEE값 : 2.303


# 미니배치 학습

$$
E = -\frac{1}{N}\sum_n\;\sum_{k}t_{nk}log\;y_{nk}
$$

- n개의 배치 데이터를 활용했을 때 바뀐 cee 공식

In [16]:
# mnist 데이터셋 로딩

from tensorflow.keras import datasets

mnist = datasets.mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()


In [17]:
X_train.shape

(60000, 28, 28)

In [18]:
y_train.shape

(60000,)

In [41]:
x = X_train.reshape(60000,-1)
x.shape

(60000, 784)

In [53]:
y_train.reshape(-1,1)

array([[5],
       [0],
       [4],
       ...,
       [5],
       [6],
       [8]], dtype=uint8)

In [48]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
encoder.fit(y_train.reshape(-1,1))
y_onehot = encoder.transform(y_train.reshape(-1,1)).toarray()
y_onehot.shape

(60000, 10)

In [49]:
y_onehot

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

## 미니배치 구현하기

In [54]:
# 훈련데이터에서 무작위로 10장만 빼오기

train_size = X_train.shape[0] # 전체 훈련 데이터셋 크기
batch_size = 10 # 미니 배치의 사이즈
batch_mask = np.random.choice(train_size,batch_size) # train_size에서 무작위로 batch_size 만큼의 정수를 선택

X_batch = X_train[batch_mask] # 랜덤으로 선택된 인덱스에 있는 데이터만 추려내기
y_batch = y_onehot[batch_mask] # 원핫 인코딩된 y_train에서 선택된 인덱스에 있는 데이터만 추려내기

print('무작위로 선택된 인덱스 : {}'.format(batch_mask))

무작위로 선택된 인덱스 : [43893 22010 16056  8873 25245 18231  3190 27958  8262 34742]


In [ ]:
# 배치 및 배치가 아닐 때 까지 고려

def cross_entropy_error(y,t):
    
    # 1차원일때에 대한 처리( 배치가 아닐 때의 처리)
    if y.ndim == 1:
        # 강제로 2차원 배열화 시키는 것
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

In [ ]:
# 원 핫 인코딩이 되어있지 않을 경우 대응

def cross_entropy_error(y,t):
    
    if y.dim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
        
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size),t])) / batch_size